In [1]:
from vietocr.tool.config import Cfg
from vietocr.model.trainer import Trainer
from only_train_once import OTO

import torch

In [2]:
config = Cfg.load_config_from_name("vgg_transformer")
config['cnn']['pretrained']=False
config['device'] = 'cpu'
config['dataloader']['num_workers'] = 0

In [3]:
dataset_params = {
    'name':'hw',
    'data_root':'./data_line/',
    'train_annotation':'train_line_annotation.txt',
    'valid_annotation':'test_line_annotation.txt'
}
config['dataset'].update(dataset_params)

In [4]:
trainer = Trainer(config=config)

Model weight /var/folders/4v/kdcdc5f15vvd8rvc67_8y6p80000gn/T/vgg_transformer.pth exsits. Ignore download!
train_hw exists. Remove folder if you want to create new dataset


/Users/lihardingnguyen/.miniconda3/envs/ai-env/lib/python3.11/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
train_hw build cluster: 100%|███████████████████████████████| 5482/5482 [00:00<00:00, 235564.08it/s]

valid_hw exists. Remove folder if you want to create new dataset



valid_hw build cluster: 100%|███████████████████████████████| 1812/1812 [00:00<00:00, 158252.55it/s]


In [5]:
len(trainer.train_gen)

172

In [35]:
dummy_input = torch.randn(32,3, 32, 300)

In [36]:
oto = OTO(model=trainer.model.cnn, dummy_input=dummy_input, strict_out_nodes=True)

OTO graph constructor
graph build


In [38]:
trainer.optimizer = oto.hesso(variant='adamw', lr=1e-5,weight_decay=1e-4,
    target_group_sparsity=0.7,
    start_pruning_step=10 * len(trainer.train_gen), 
    pruning_periods=10,
    pruning_steps=10 * len(trainer.train_gen))

Setup HESSO
Target redundant groups per period:  [385, 385, 385, 385, 385, 385, 385, 385, 385, 387]


In [ ]:
trainer.train()